In [ ]:
import torch
import torchvision.transforms as transforms
from torchvision.models import resnet50
from PIL import Image
import numpy as np
import faiss
import pickle
import psycopg2


In [ ]:
# Load pre-trained ResNet-50 model
model = resnet50(pretrained=True)
model = model.eval()  # Set model to evaluation mode

# Remove the final fully connected layer to use ResNet as a feature extractor
model = torch.nn.Sequential(*(list(model.children())[:-1]))

# Image preprocessing (same as used for ImageNet pretraining)
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


In [ ]:
# Load the FAISS index
faiss_index_path = 'faiss_ivf_hnsw_index.index'
index = faiss.read_index(faiss_index_path)

# Load the faiss_id_to_image_name mapping from pickle file
with open('faiss_id_to_image_name.pkl', 'rb') as f:
    faiss_id_to_image_name = pickle.load(f)

print("FAISS index and mappings loaded successfully.")


In [ ]:
# Function to extract features from the query image
def extract_query_feature(image_path):
    image = Image.open(image_path).convert('RGB')
    image = preprocess(image).unsqueeze(0)  # Add batch dimension
    with torch.no_grad():
        feature_vector = model(image)
    return feature_vector.squeeze().numpy()  # Convert to NumPy array and remove batch dimension

# Example: Path to the query image
query_image_path = 'path_to_query_image.jpg'

# Extract feature vector from the query image
query_feature_vector = extract_query_feature(query_image_path)
query_feature_vector = np.array([query_feature_vector]).astype('float32')  # Reshape for FAISS

print("Query image feature extracted.")


In [ ]:
# Number of nearest neighbors to retrieve
k = 1  # You can retrieve more than one if needed

# Search FAISS index
D, I = index.search(query_feature_vector, k)

# If a similar image is found
if I[0][0] != -1:  # FAISS returns -1 if no match is found
    faiss_index_id = I[0][0]
    similar_image_name = faiss_id_to_image_name[faiss_index_id]
    print(f"Found similar image: {similar_image_name}")
else:
    print("No similar images found.")


In [ ]:
# PostgreSQL database connection details
db_name = 'your_db_name'
db_user = 'your_db_user'
db_password = 'your_password'
db_host = 'localhost'  # or the IP address if hosted externally
db_port = '5432'

# Connect to the PostgreSQL database
conn = psycopg2.connect(dbname=db_name, user=db_user, password=db_password, host=db_host, port=db_port)

# Create a cursor to execute queries
cur = conn.cursor()

# Query the database using the image name (primary key)
query = "SELECT * FROM images_table WHERE image_name = %s;"
cur.execute(query, (similar_image_name,))

# Fetch the result
result = cur.fetchone()

# Close the cursor and connection
cur.close()
conn.close()

# Display the result
if result:
    print(f"Database result: {result}")
else:
    print("No data found for the similar image in the database.")
